In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 09 14:32:26 2018
@author: 20914
"""

from WindPy import w
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
rc('mathtext', default='regular')
import math
from scipy.stats import mstats
from datetime import datetime
from pandas import DataFrame
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [100]:
def get_data(code_dir, start, end):
    pre_start=w.tdaysoffset(-2, start, "Period=Y").Data[0][0]
    data = w.wsd(code_dir, "pre_close,high,low,close,lastradeday_s", pre_start, end)
    df = pd.DataFrame(data.Data, index=data.Fields).T
    df['DailyReturn'] = df['CLOSE'] / df['PRE_CLOSE']    
    df['LASTRADEDAY_S'] = pd.to_datetime(df['LASTRADEDAY_S'])
    df.set_index('LASTRADEDAY_S', inplace=True)
    
    return df

In [109]:
def calculate_vol(data, start, end, method):

    periods = {'1Week': 5, '2Week': 10, '1Month': 21, '2Month': 41, '3Month': 62, '4Month': 83, 
               '5Month': 104, '6Month': 124, '9Month': 186, '1Year': 249, '2Year': 497}#要计算的历史波动率的窗口大小
    ## ewma
    if method == 'EWMA':
        data['LOG_2'] = data['DailyReturn'].apply(lambda x: (np.log(x))**2)
        for key, value in periods.items():
            data[key] = data['LOG_2'].ewm(span= value ).mean()** 0.2 * 252**0.5

    ## cla
    elif method == 'CLA': 
        data['LOG'] = data['DailyReturn'].apply(lambda x: np.log(x))
        for key, value in periods.items():
            data[key] = data['LOG'].rolling(window=value).std() * 252**0.5

    ## Parkinson
    elif method == 'PARKINSON': 
        data['High_to_low'] = data['HIGH'] / data['LOW'] ** 2 / (4 * np.log(2))
        for key, value in periods.items():
            data[key] = data['High_to_low'].rolling(window=value).mean() * 252**0.5

    return data.loc[start:end]